# 이미지에서 인식된 재료 벡터와 각 레시피 재료 벡터 사이를 계산

In [1]:
import torch
import torch.nn as nn
import pandas as pd
from ultralytics import YOLO
import ast

In [2]:
def emb_func(ing_list):

    a = [0] * 83
    for k in ing_list:
        a[k] = 1

    return a

In [3]:
# 모델 불러오기
model = YOLO('../data/CASE7.pt')

In [4]:
# cosine similarity class 선언
cos = nn.CosineSimilarity(dim=1)

In [5]:
recipe_df = pd.read_pickle('../data/embedded_only_robo_df.pkl')
recipe_df

,id,name,new_ingredients,minutes,steps,description,vege/vegan,food type,healthiness,new_ingredients_idx,emb_num
0,23933,chinese candy,"[butter, salt, noodle]",15,['melt butterscotch chips in heavy saucepan ov...,"a little different, and oh so good. i include ...",non-vegan,dish,unhealthy,"[12, 71, 54]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,111875,the elvis smoothie,"[butter, milk, banana, sugar]",15,"['place all ingredients in blender', 'blend un...","this is a great smoothie, thick and creamy. my...",non-vegan,dish,healthy,"[12, 52, 5, 76]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,101115,no name beef recipe loco moco,"[rice, beef, egg, onion]",30,"['cook rice according to package directions', ...",i saw this dish on a tv travel channel. it was...,non-vegan,dish,healthy,"[69, 7, 32, 57]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3,163238,the creamed dried beef chipped beef recipe,"[beef, butter, flour, milk, pepper]",75,"['melt butter in 2 quart sauce pan or larger',...",this is a long-time favorite recipe of our lan...,non-vegan,dish,healthy,"[7, 12, 37, 52, 62]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, ..."
4,16327,creamed spinach,"[onion, cheese, yogurt, spinach]",15,['i keep drained yogurt available in my fridge...,i created this recipe one evening when i wante...,non-vegan,dish,unhealthy,"[57, 19, 81, 73]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...
2640,326105,zucchini with bacon corn peppers,"[garlic, onion, cheese, zucchini, bacon, corn,...",45,"['in a wide frying pan , cook your bacon until...",this is a very colorful addition to any meal. ...,non-vegan,dish,unhealthy,"[38, 57, 19, 82, 4, 29, 62]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2641,54175,zucchini with green peppercorns lemon and par...,"[lemon, salt, cheese, zucchini, oil, pepper]",15,"['whisk together the lemon juice , olive oil ,...",saw this on foodtv's cooking thin show. it is ...,non-vegan,dish,unhealthy,"[47, 71, 19, 82, 56, 62]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2642,320622,zucchini with jalapeno monterey jack,"[zucchini, flour, pepper]",15,['spread the zucchini in a microwave-safe dish...,simple and yummy!,non-vegan,dish,unhealthy,"[82, 37, 62]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2643,152200,zucchini with parmesan,"[salt, cheese, zucchini, oil, pepper]",40,['heat oil in a large skillet over medium heat...,"this recipe came from ""eating well"" magazine, ...",non-vegan,dish,unhealthy,"[71, 19, 82, 56, 62]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [7]:
recipe_df

,id,name,new_ingredients,minutes,steps,description,vege/vegan,food type,healthiness,new_ingredients_idx,emb_num
0,23933,chinese candy,"[butter, salt, noodle]",15,['melt butterscotch chips in heavy saucepan ov...,"a little different, and oh so good. i include ...",non-vegan,dish,unhealthy,"[12, 71, 54]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,111875,the elvis smoothie,"[butter, milk, banana, sugar]",15,"['place all ingredients in blender', 'blend un...","this is a great smoothie, thick and creamy. my...",non-vegan,dish,healthy,"[12, 52, 5, 76]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,101115,no name beef recipe loco moco,"[rice, beef, egg, onion]",30,"['cook rice according to package directions', ...",i saw this dish on a tv travel channel. it was...,non-vegan,dish,healthy,"[69, 7, 32, 57]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3,163238,the creamed dried beef chipped beef recipe,"[beef, butter, flour, milk, pepper]",75,"['melt butter in 2 quart sauce pan or larger',...",this is a long-time favorite recipe of our lan...,non-vegan,dish,healthy,"[7, 12, 37, 52, 62]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, ..."
4,16327,creamed spinach,"[onion, cheese, yogurt, spinach]",15,['i keep drained yogurt available in my fridge...,i created this recipe one evening when i wante...,non-vegan,dish,unhealthy,"[57, 19, 81, 73]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...
2640,326105,zucchini with bacon corn peppers,"[garlic, onion, cheese, zucchini, bacon, corn,...",45,"['in a wide frying pan , cook your bacon until...",this is a very colorful addition to any meal. ...,non-vegan,dish,unhealthy,"[38, 57, 19, 82, 4, 29, 62]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2641,54175,zucchini with green peppercorns lemon and par...,"[lemon, salt, cheese, zucchini, oil, pepper]",15,"['whisk together the lemon juice , olive oil ,...",saw this on foodtv's cooking thin show. it is ...,non-vegan,dish,unhealthy,"[47, 71, 19, 82, 56, 62]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2642,320622,zucchini with jalapeno monterey jack,"[zucchini, flour, pepper]",15,['spread the zucchini in a microwave-safe dish...,simple and yummy!,non-vegan,dish,unhealthy,"[82, 37, 62]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2643,152200,zucchini with parmesan,"[salt, cheese, zucchini, oil, pepper]",40,['heat oil in a large skillet over medium heat...,"this recipe came from ""eating well"" magazine, ...",non-vegan,dish,unhealthy,"[71, 19, 82, 56, 62]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [8]:
type(recipe_df['emb_num'])

pandas.core.series.Series

In [9]:
recipe_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2645 entries, 0 to 2644
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   2645 non-null   int64 
 1   name                 2645 non-null   object
 2   new_ingredients      2645 non-null   object
 3   minutes              2645 non-null   int64 
 4   steps                2645 non-null   object
 5   description          2645 non-null   object
 6   vege/vegan           2645 non-null   object
 7   food type            2645 non-null   object
 8   healthiness          2645 non-null   object
 9   new_ingredients_idx  2645 non-null   object
 10  emb_num              2645 non-null   object
dtypes: int64(2), object(9)
memory usage: 227.4+ KB


In [10]:
def run(image_dir):
    global model
    global cos
    global recipe_df
    

    print('predicting ingredients labels...\n')
    # image = cv2.imshow(image_dir)
    result = model.predict(image_dir)

    # print( 'result : ' , result )

    img_ing_label = result[0].boxes.cls.unique()       # tensor, float tensor([1.,2.,3.])
    # print( img_ing_label )

    img_ing_label = img_ing_label.tolist()            # list, float [1.,2.,3.]
    img_ing_label = [int(k) for k in img_ing_label]   # list, int   [1,2,3]

    ing_image = torch.tensor(emb_func(img_ing_label)) # embedding tensor tensor([0,1,1,1,0,...,0])
                                                      # emb_func의 결과 list를 cosine similarity 계산을 위해 텐서로 바꿈
                                                      # tensor shape (84) -> dim=1으로 similarity 계산을 해야하므로 (1,83)로 바꿔줘야 함
    # print( ing_image.shape )
    ing_image = ing_image.reshape(1,-1)               # tensor.reshape(1,-1) # dimension에 숫자를 넣으면 그 숫자로 dimension 맞춰주고,
                                                      # -1을 넣으면 알아서 해달라는 의미. 따라서 dim=0은 1, dim=1은 알아서 (84)
    # print( ing_image.shape )
    # print( recipe_df.iloc[ : , -1] )
    # return
    ing_recipe = []
    for string_list in recipe_df['emb_num']:
        if type(string_list) == 'string':
            ing_recipe.append( ast.literal_eval( string_list ) ) # "[ 0 , 1, 0 , 0]" -> [ 0 , 1, 0 , 0 ]
        else:
            ing_recipe.append( list(string_list) ) # numpy -> list
                            
        # print( ast.literal_eval(string_list) )
        # break
    # print( ing_recipe)
    # ing_recipe = [ast.literal_eval(string_list) for string_list in recipe_df['emb_num'] ] # 이 부분 필요 없을수도 있음 [0,1,1,1,0,...,0] [1,2,3]
    # ing_recipe = torch.tensor(emb_func(ing_recipe)) # [1,2,3] -> [0,1,1,1,0,...,0] 으로 바꿔주는 과정.
    # ing_recipe = recipe_df['ing'] # [0,1,1,1,0,...,0] 들의 모임. data 갯수만큼
    # ing_recipe = (156,84) Series (list 개념? dataframe의 column 하나만 가져온 것. tensor는 아님.)

    ing_recipe = torch.tensor(ing_recipe) # tensor. dim = (156,83)

    # torch method들은 대부분 deep learning에서 사용되는 개념
    # data type이 float64 이게 기본
    # 따라서 계산한 embedding인 ing_image, ing_recipe를 int -> float으로 바꾼 뒤 cosine similarity 계산

    ing_image = ing_image.type(torch.float64)
    ing_recipe = ing_recipe.type(torch.float64)

    similarity = cos(ing_image, ing_recipe ) # similarity가 저장된 tensor dim=(156, 1)
    # print( 'sim' , similarity.shape )
    # return 
    recipe_df['similarity'] = similarity # recipe들이랑 입력한 image랑 similarity를 가지고있는 dataframe = recipe_df

    result_df = recipe_df.sort_values(by=['similarity', 'steps'], axis=0, ascending=[False, True]).iloc[:3,:] # n_steps -> steps

    print('===========================================================')
    print(f'1st recipe : {result_df.iloc[0, 1]} , similarity : {result_df.iloc[0, -1]}')
    print(f'2nd recipe : {result_df.iloc[1, 1]} , similarity : {result_df.iloc[1, -1]}')
    print(f'3rd recipe : {result_df.iloc[2, 1]} , similarity : {result_df.iloc[2, -1]}')
    print('\n')
    rank = input('show detail recipe : ')
    # rank = 1
    n = int(rank)-1
    
    show_recipe = {
       'name': result_df.iloc[n,1],
       'time': result_df.iloc[n, 3],
       'steps': result_df.iloc[n, 4],
       'description': result_df.iloc[n, 5],
       'vegan': result_df.iloc[n, 6],
       'food_type': result_df.iloc[n, 7]
    }

    print(f"🏆 순위 {rank}위 레시피 이름은 '{show_recipe['name']}'입니다.\n"
          f"⏰ 조리 시간은 약 {show_recipe['time']}분이고,\n" # -> time
          f"🌍 {show_recipe['vegan']}도 드실 수 있는 레시피입니다. \n" # vega/vegan -> vegan
          f"🍽️ 참고로 이 레시피는 {show_recipe['food_type']} 입니다.") # food type -> food_type

    # print(show_recipe)
    # print(f"rank {rank} recipe name is {show_recipe['name']}.\n"
    #       f"칼로리는 {show_recipe['calories']}이고, ~~~~{}")

    # print(show_recipe)


if __name__ == '__main__':
    image_dir = input('image directory 입력 : ')
    # image_dir = 'test.jpg'
    run(image_dir)

image directory 입력 : C:/Users/miy/Documents/ds_study/data/test2/images/my_fridge_l.jpeg
predicting ingredients labels...




image 1/1 C:\Users\miy\Documents\ds_study\data\test2\images\my_fridge_l.jpeg: 256x320 1 tomato, 6 yogurts, 590.4ms
Speed: 2.0ms preprocess, 590.4ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 320)


1st recipe : lebanese laban  yogurt , similarity : 0.4999999999999999
2nd recipe : custom fruit salad , similarity : 0.408248290463863
3rd recipe : easy smoothies , similarity : 0.408248290463863


show detail recipe : 1
🏆 순위 1위 레시피 이름은 'lebanese laban  yogurt'입니다.
⏰ 조리 시간은 약 20분이고,
🌍 non-vegan도 드실 수 있는 레시피입니다. 
🍽️ 참고로 이 레시피는 dish 입니다.


---------------------------

* 이미지 경로 입력할 때는 '' 안 쓰고 절대경로로.
ex. C:/Users/miy/Documents/ds_study/data/test/images/DSC_6089_JPG_jpg.rf.8fdb83079ef640569866a9e2d34a0f68.jpg

In [ ]:
def run(image_dir):
    global model
    global cos
    global recipe_df
    

    print('predicting ingredients labels...\n')
    # image = cv2.imshow(image_dir)
    result = model.predict(image_dir)

    # print( 'result : ' , result )

    img_ing_label = result[0].boxes.cls.unique()       # tensor, float tensor([1.,2.,3.])
    # print( img_ing_label )

    img_ing_label = img_ing_label.tolist()            # list, float [1.,2.,3.]
    img_ing_label = [int(k) for k in img_ing_label]   # list, int   [1,2,3]

    ing_image = torch.tensor(emb_func(img_ing_label)) # embedding tensor tensor([0,1,1,1,0,...,0])
                                                      # emb_func의 결과 list를 cosine similarity 계산을 위해 텐서로 바꿈
                                                      # tensor shape (84) -> dim=1으로 similarity 계산을 해야하므로 (1,83)로 바꿔줘야 함
    # print( ing_image.shape )
    ing_image = ing_image.reshape(1,-1)               # tensor.reshape(1,-1) # dimension에 숫자를 넣으면 그 숫자로 dimension 맞춰주고,
                                                      # -1을 넣으면 알아서 해달라는 의미. 따라서 dim=0은 1, dim=1은 알아서 (84)
    # print( ing_image.shape )
    # print( recipe_df.iloc[ : , -1] )
    # return
    ing_recipe = []
    for string_list in recipe_df['emb_num']:
        if type(string_list) == 'string':
            ing_recipe.append( ast.literal_eval( string_list ) ) # "[ 0 , 1, 0 , 0]" -> [ 0 , 1, 0 , 0 ]
        else:
            ing_recipe.append( list(string_list) ) # numpy -> list
                            
        # print( ast.literal_eval(string_list) )
        # break
    # print( ing_recipe)
    # ing_recipe = [ast.literal_eval(string_list) for string_list in recipe_df['emb_num'] ] # 이 부분 필요 없을수도 있음 [0,1,1,1,0,...,0] [1,2,3]
    # ing_recipe = torch.tensor(emb_func(ing_recipe)) # [1,2,3] -> [0,1,1,1,0,...,0] 으로 바꿔주는 과정.
    # ing_recipe = recipe_df['ing'] # [0,1,1,1,0,...,0] 들의 모임. data 갯수만큼
    # ing_recipe = (156,84) Series (list 개념? dataframe의 column 하나만 가져온 것. tensor는 아님.)

    ing_recipe = torch.tensor(ing_recipe) # tensor. dim = (156,83)

    # torch method들은 대부분 deep learning에서 사용되는 개념
    # data type이 float64 이게 기본
    # 따라서 계산한 embedding인 ing_image, ing_recipe를 int -> float으로 바꾼 뒤 cosine similarity 계산

    ing_image = ing_image.type(torch.float64)
    ing_recipe = ing_recipe.type(torch.float64)

    similarity = cos(ing_image, ing_recipe ) # similarity가 저장된 tensor dim=(156, 1)
    # print( 'sim' , similarity.shape )
    # return 
    recipe_df['similarity'] = similarity # recipe들이랑 입력한 image랑 similarity를 가지고있는 dataframe = recipe_df

    result_df = recipe_df.sort_values(by=['similarity', 'steps'], axis=0, ascending=[False, True]).iloc[:3,:] # n_steps -> steps

    print('===========================================================')
    print(f'1st recipe : {result_df.iloc[0, 1]} , similarity : {result_df.iloc[0, -1]}')
    print(f'2nd recipe : {result_df.iloc[1, 1]} , similarity : {result_df.iloc[1, -1]}')
    print(f'3rd recipe : {result_df.iloc[2, 1]} , similarity : {result_df.iloc[2, -1]}')
    print('\n')
    rank = input('show detail recipe : ')
    # rank = 1
    n = int(rank)-1
    
    show_recipe = {
       'name': result_df.iloc[n,1],
       'time': result_df.iloc[n, 3],
       'steps': result_df.iloc[n, 4],
       'description': result_df.iloc[n, 5],
       'vegan': result_df.iloc[n, 6],
       'food_type': result_df.iloc[n, 7]
    }

    print(f"🏆 순위 {rank}위 레시피는 {show_recipe['name']}입니다.\n"
          f"⏰ 조리 시간은 {show_recipe['time']}이고,\n" # -> time
          f"🌍 {show_recipe['vegan']}도 드실 수 있는 레시피입니다. \n" # vega/vegan -> vegan
          f"🍽️ 참고로 이 레시피는 {show_recipe['food_type']} 입니다.") # food type -> food_type

    # print(show_recipe)
    # print(f"rank {rank} recipe name is {show_recipe['name']}.\n"
    #       f"칼로리는 {show_recipe['calories']}이고, ~~~~{}")

    # print(show_recipe)


if __name__ == '__main__':
    image_dir = input('image directory 입력 : ')
    # image_dir = 'test.jpg'
    run(image_dir)
    

In [ ]:
# 구 코드 - 에러 남 => 위 코드로 해결
def run(image_dir):
    global model
    global cos
    global recipe_df

    print('predicting ingredients labels...\n')
    result = model.predict(image_dir)

    img_ing_label = result[0].boxes.cls.unique()       # tensor, float tensor([1.,2.,3.])

    img_ing_label = img_ing_label.tolist()            # list, float [1.,2.,3.]
    img_ing_label = [int(k) for k in img_ing_label]   # list, int   [1,2,3]

    ing_image = torch.tensor(emb_func(img_ing_label)) # embedding tensor tensor([0,1,1,1,0,...,0])
                                                      # emb_func의 결과 list를 cosine similarity 계산을 위해 텐서로 바꿈
                                                      # tensor shape (84) -> dim=1으로 similarity 계산을 해야하므로 (1,84)로 바꿔줘야 함
    ing_image = ing_image.reshape(1,-1)               # tensor.reshape(1,-1) # dimension에 숫자를 넣으면 그 숫자로 dimension 맞춰주고,
                                                      # -1을 넣으면 알아서 해달라는 의미. 따라서 dim=0은 1, dim=1은 알아서 (84)

    ing_recipe = [ast.literal_eval(string_list) for string_list in recipe_df['emb_num']] # 이 부분 필요 없을수도 있음 [0,1,1,1,0,...,0] [1,2,3]
    # ing_recipe = torch.tensor(emb_func(ing_recipe)) # [1,2,3] -> [0,1,1,1,0,...,0] 으로 바꿔주는 과정.
    # ing_recipe = recipe_df['ing'] # [0,1,1,1,0,...,0] 들의 모임. data 갯수만큼
    # ing_recipe = (156,84) Series (list 개념? dataframe의 column 하나만 가져온 것. tensor는 아님.)

    ing_recipe = torch.tensor(ing_recipe) # tensor. dim = (156,84)

    # torch method들은 대부분 deep learning에서 사용되는 개념
    # data type이 float64 이게 기본
    # 따라서 계산한 embedding인 ing_image, ing_recipe를 int -> float으로 바꾼 뒤 cosine similarity 계산

    ing_image = ing_image.type(torch.float64)
    ing_recipe = ing_recipe.type(torch.float64)

    similarity = cos(ing_image, ing_recipe) # similarity가 저장된 tensor dim=(156, 1)

    recipe_df['similarity'] = similarity # recipe들이랑 입력한 image랑 similarity를 가지고있는 dataframe = recipe_df

    result_df = recipe_df.sort_values(by=['similarity', 'n_steps'], axis=0, ascending=[False, True]).iloc[:3,:]

    print('===========================================================')
    print(f'1st recipe : {result_df.iloc[0, 1]} , similarity : {result_df.iloc[0, -1]}')
    print(f'2nd recipe : {result_df.iloc[1, 1]} , similarity : {result_df.iloc[1, -1]}')
    print(f'3rd recipe : {result_df.iloc[2, 1]} , similarity : {result_df.iloc[2, -1]}')
    print('\n')
    rank = input('show detail recipe : ')

    n = int(rank)-1
    show_recipe = {
        'name': result_df.iloc[n,2],
        'time': result_df.iloc[n, 4],
        'steps': result_df.iloc[n, 5],
        'description': result_df.iloc[n, 6],
        'vegan': result_df.iloc[n, 15],
        'food_type': result_df.iloc[n, 16]
    }

    # print(f"rank {rank} recipe name is {show_recipe['name']}.\n"
    #       f"칼로리는 {show_recipe['calories']}이고, ~~~~{}")

    print(show_recipe)


if __name__ == '__main__':
    image_dir = input('image directory 입력 : ')

    run(image_dir)
    

In [ ]:
aaa = {'0' : 'aaa', '1' : 'bbb'}

list(aaa.values()).index('bbb')